In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg



In [2]:
# Database initialization
conn = sqlite3.connect('bmi_data.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS bmi_records
             (id INTEGER PRIMARY KEY AUTOINCREMENT, date TEXT, weight REAL, height REAL, bmi REAL)''')
conn.commit()



In [3]:
def calculate_bmi(weight, height):
    try:
        bmi = weight / (height ** 2)
        return round(bmi, 2)
    except ZeroDivisionError:
        return None



In [4]:
def classify_bmi(bmi):
    if bmi is None:
        return "Error: Height should be greater than 0."
    elif bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal weight"
    elif 25 <= bmi < 30:
        return "Overweight"
    else:
        return "Obese"


In [5]:

def save_data(date, weight, height, bmi):
    c.execute("INSERT INTO bmi_records (date, weight, height, bmi) VALUES (?, ?, ?, ?)", (date, weight, height, bmi))
    conn.commit()



In [6]:
def plot_bmi_trend():
    c.execute("SELECT date, bmi FROM bmi_records")
    data = c.fetchall()

    if not data:
        messagebox.showinfo("No Data", "No BMI data available for plotting.")
        return

    dates, bmis = zip(*data)

    fig, ax = plt.subplots()
    ax.plot(dates, bmis, marker='o', linestyle='-')
    ax.set(xlabel='Date', ylabel='BMI', title='BMI Trend Over Time')
    ax.grid()
    
    plt.xticks(rotation=45)
    
    canvas = FigureCanvasTkAgg(fig, master=window)
    canvas_widget = canvas.get_tk_widget()
    canvas_widget.grid(row=5, column=0, columnspan=3)



In [7]:
def calculate_and_display():
    try:
        weight = float(weight_entry.get())
        height = float(height_entry.get())
    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter valid numerical values for weight and height.")
        return

    bmi = calculate_bmi(weight, height)
    category = classify_bmi(bmi)

    result_label.config(text=f"Your BMI is: {bmi}\nCategory: {category}")

    date = date_entry.get()
    save_data(date, weight, height, bmi)

    plot_bmi_trend()



In [8]:
# GUI setup
window = tk.Tk()
window.title("BMI Calculator")



''

In [9]:
# Entry fields
date_label = ttk.Label(window, text="Date:")
date_label.grid(row=0, column=0, sticky='e')

date_entry = ttk.Entry(window)
date_entry.grid(row=0, column=1)

weight_label = ttk.Label(window, text="Weight (kg):")
weight_label.grid(row=1, column=0, sticky='e')

weight_entry = ttk.Entry(window)
weight_entry.grid(row=1, column=1)

height_label = ttk.Label(window, text="Height (m):")
height_label.grid(row=2, column=0, sticky='e')

height_entry = ttk.Entry(window)
height_entry.grid(row=2, column=1)



In [10]:
# Calculate button
calculate_button = ttk.Button(window, text="Calculate BMI", command=calculate_and_display)
calculate_button.grid(row=3, column=0, columnspan=2)



In [11]:
# Result label
result_label = ttk.Label(window, text="")
result_label.grid(row=4, column=0, columnspan=2)



In [ ]:
# Run the GUI
window.mainloop()



In [13]:
# Close the database connection
conn.close()